In [ ]:
#import libraries
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter
import shutil
from distutils.dir_util import copy_tree

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

# Split directories to shuffle photos directly

In [ ]:
dataset_dir = '/kaggle/input/dataset/training'

training_dir = '/kaggle/working/training'
validation_dir = '/kaggle/working/validation'

os.mkdir(training_dir)
os.mkdir(validation_dir)

copy_tree(dataset_dir, training_dir)

In [ ]:
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Defining input_shape, batch_size and epochs

In [ ]:
input_shape = (256, 256, 3)
batch_size = 16
epochs = 100

# Folder split + shuffle

In [ ]:
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    
for subdir, dirs, files in os.walk(dataset_dir):
    for class_dir in dirs:
        filenames = os.listdir(os.path.join(training_dir, class_dir))
        np.random.shuffle(filenames)
        split = int(0.8 * len(filenames))
        train_filenames = filenames[:split]
        valid_filenames = filenames[split:]
        if not os.path.exists(os.path.join(validation_dir, class_dir)):
            os.mkdir(os.path.join(validation_dir, class_dir))
        for file in valid_filenames:
            shutil.move(os.path.join(training_dir, class_dir, file), os.path.join(validation_dir, class_dir, file))

# Data Augmentation

Different augmentations for different fine tunings

First: rotation, zoom, shifts, horizontal flip

Second: shear, brightness, rotation, zoom, vertical flip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rotation_range=30,
                                    width_shift_range=25,
                                    height_shift_range=25,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=False,
                                    fill_mode='reflect',
                                    rescale=1./255)

new_augmentation = ImageDataGenerator(rotation_range=80,
                                    zoom_range=0.5,
                                    shear_range=0.1,
                                    brightness_range=[0.3, 1.6],  
                                    horizontal_flip=False,
                                    vertical_flip=True,
                                    fill_mode='reflect',
                                    rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)

Only shuffle training set(s)

In [ ]:
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(256, 256),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=seed)

new_train_gen = new_augmentation.flow_from_directory(training_dir,
                                               target_size=(256, 256),
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=seed)


valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size=(256, 256),
                                               batch_size=batch_size, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=seed)

# Importing Xception

In [ ]:
transfer_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=input_shape, pooling='max')

# Weighted Classes

In [ ]:
counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()} 
print(class_weights)

In [ ]:
transfer_model.summary()
transfer_model.layers

Setting the supernet as non-trainable

In [ ]:
for layer in transfer_model.layers:    
    layer.trainable = False

# Connecting the net

In [ ]:
model = tf.keras.Sequential()
model.add(transfer_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.2, noise_shape=None, seed=seed))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=14, activation='softmax'))

model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')

model.summary()

# Callbacks and Early Stopping

Patience: 10

monitor: val_loss

In [ ]:
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('experiments_data')

  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')
  
  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))

  if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
      
  callbacks = []

  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts') 
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)


  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                     save_weights_only=False,
                                                     save_best_only=False)

  
  callbacks.append(ckpt_callback)


  # Early Stopping
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

# First Training

In [ ]:
model_callbacks = create_folders_and_callbacks(model_name='model')

tl_history = model.fit(x=train_gen,
                      epochs=epochs,
                      steps_per_epoch=len(train_gen),
                      validation_data=valid_gen,
                      validation_steps=len(valid_gen),
                      class_weight=class_weights,
                      callbacks=model_callbacks).history

Plotting accuracy of the first model

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(tl_history['accuracy'], label='Training', alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Validation', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model.save("xception_tl_classweights")

In [ ]:
from tensorflow import keras
model = keras.models.load_model("xception_tl_classweights")
model.summary()

# Fine Tuning: freezing supernet layers

Important: lower learning rate (1e-4)

In [ ]:
freeze_until = 126

for layer in model.layers[0].layers[:freeze_until]:    
    layer.trainable = False
    
for layer in model.layers[0].layers[freeze_until:]:
    layer.trainable = True
    print(layer)
    
for layer in model.layers[0].layers:
    print(layer, layer.trainable)
    
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

model.summary()

In [ ]:
xception_callbacks = create_folders_and_callbacks('XCEPTION_1FT')

ft_history = model.fit(x=train_gen,
                      epochs=epochs,
                      steps_per_epoch=len(train_gen),
                      validation_data=valid_gen,
                      validation_steps=len(valid_gen),
                      class_weight=class_weights,
                      callbacks=xception_callbacks).history

In [ ]:
model.save("xception_tl_ft_classweights_finale")

Plotting accuracy of the orginal model and the one with fine tuning

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#ff7f0e')
plt.plot(ft_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model = keras.models.load_model("xception_tl_ft_classweights_finale")
model.summary()

# Second Fine Tuning: freeze a little more

Important: lower learning rate (1e-5)

In [ ]:
freeze_until = 129

for layer in model.layers[0].layers[:freeze_until]:    
    layer.trainable = False
    
for layer in model.layers[0].layers[freeze_until:]:
    layer.trainable = True
    print(layer)
    
for layer in model.layers[0].layers:
    print(layer, layer.trainable)
    
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

model.summary()

Second fine tuning uses new_train_gen -> different augmentation

In [ ]:
xception_callbacks = create_folders_and_callbacks('XCEPTION_2FT')

second_ft_history = model.fit(x=new_train_gen,
                      epochs=epochs,
                      steps_per_epoch=len(new_train_gen),
                      validation_data=valid_gen,
                      validation_steps=len(valid_gen),
                      class_weight=class_weights,
                      callbacks=xception_callbacks).history

In [ ]:
model.save("xception_tl_2ft_classweights_finale")

Final plotting of the three trained models

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#ff7f0e')
plt.plot(ft_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#4D61E2')
plt.plot(second_ft_history['accuracy'], alpha=.3, color='#7E1E9C', linestyle='--')
plt.plot(second_ft_history['val_accuracy'], label='Second Fine Tuning', alpha=.8, color='#7E1E9C')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()